<a href="https://colab.research.google.com/github/UmbertoB/neural-networks-research/blob/main/research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2, VGG16
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers
from keras.regularizers import l2
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten, Dense, Activation, Dropout, \
    GlobalAveragePooling2D
from tensorflow.keras.utils import plot_model
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import numpy as np

def compile_model(layers, alpha):
    model = models.Sequential()

    for layer in layers:
        model.add(layer)

    opt = Adam(learning_rate=alpha)

    model.build()
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['acc'])

    print("Modelo Compilado...")
    return model


def data_augment(dataset_path, batch_size, input_shape):
    image_gen = ImageDataGenerator(rotation_range=40, rescale=1 / 255, horizontal_flip=True, vertical_flip=True)

    print(dataset_path)
    train_images = image_gen.flow_from_directory(dataset_path + '/train', target_size=input_shape[:2],
                                                 batch_size=batch_size, class_mode='binary')
    validation_images = image_gen.flow_from_directory(dataset_path + '/validation',
                                                      target_size=input_shape[:2], batch_size=batch_size,
                                                      class_mode='binary')
    test_images = image_gen.flow_from_directory(dataset_path + '/test', target_size=input_shape[:2],
                                                batch_size=batch_size, class_mode='binary')

    return train_images, validation_images, test_images


def create_callbacks(alpha):
    filepath = "model_one.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta=alpha, patience=5, verbose=1)
    erl_stopping = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', verbose=1)

    callbacks = [checkpoint, lr_reduce, erl_stopping]
    return callbacks


def generate_model(dataset_path, input_shape, batch_size, alpha, epoch, layers):
    model = compile_model(layers, alpha)
    callbacks = create_callbacks(alpha)
    train_images, validation_images, test_images = data_augment(dataset_path, batch_size, input_shape)

    print("Iniciando treino do Modelo...")
    history = model.fit(
        train_images,
        validation_data=validation_images,
        callbacks=callbacks,
        epochs=epoch)

    model.evaluate(test_images)

    return history, model


bs = 8
alp = 1e-3
eph = 30
i_shape = (200, 200, 3)
lyrs = [Flatten(input_shape=i_shape),
        Dense(128),
        Dense(1, activation='sigmoid')]

generate_model('drive/MyDrive/DATASET', i_shape, bs, alp, eph, lyrs)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 120000)            0         
                                                                 
 dense_3 (Dense)             (None, 128)               15360128  
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 15,360,257
Trainable params: 15,360,257
Non-trainable params: 0
_________________________________________________________________
Modelo Compilado...
drive/MyDrive/DATASET
Found 2100 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Iniciando treino do Modelo...
Epoch 1/30
263/263 [==============================] - ETA: 0s - loss: 13.8843 - acc: 0.6786
Epoch 1: val_acc i